In [1]:
import os
import numpy as np
from dotenv import load_dotenv
from langchain_community.graphs import Neo4jGraph
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.embeddings import HuggingFaceEmbeddings
from tqdm import tqdm
import warnings
from IPython.display import display, HTML, Markdown
import re
import time
import pandas as pd
import json
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, matthews_corrcoef
warnings.filterwarnings('ignore')
warnings.filterwarnings("ignore", category=UserWarning, module="neo4j")
import logging
load_dotenv()

True

In [2]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [3]:
NEO4j_URI = 'bolt://localhost:7687'
NEO4j_USER = 'neo4j'
NEO4j_PASSWORD = os.environ.get('pass')
graph = Neo4jGraph(url=NEO4j_URI, username=NEO4j_USER, password=NEO4j_PASSWORD)

In [4]:
def retrieve_similar_events(query_text, graph, embeddings, top_k=5, embedding_weight=0.6, structure_weight=0.4, similarity_threshold=0.5):
    """
    Retrieve similar events using a hybrid approach with optimized query performance.
    """
    query_embedding = embeddings.embed_query(query_text)
    
    cypher_query = """
    MATCH (e:Event)
    WHERE e.embedding IS NOT NULL AND e.text IS NOT NULL
    
    // Efficient connection counting using COUNT
    OPTIONAL MATCH (e)-[:RESULTS_IN]->(effect:Effect)
    WITH e, COUNT(effect) AS effect_count
    
    OPTIONAL MATCH (e)<-[:CAUSES]-(cause:Cause)
    WITH e, effect_count, COUNT(cause) AS cause_count
    
    OPTIONAL MATCH (e)-[:HAS_TRIGGER]->(trigger:Trigger)
    WITH e, effect_count, cause_count, COUNT(trigger) AS trigger_count
    
    // Calculate structural score (maintain original binary approach)
    WITH e, effect_count, cause_count, trigger_count,
         CASE 
             WHEN effect_count + cause_count + trigger_count = 0 
             THEN 0.0 
             ELSE 1.0 
         END AS structural_score
    
    // Calculate similarity and hybrid score
    WITH e, effect_count, cause_count, trigger_count, structural_score,
         gds.similarity.cosine(e.embedding, $query_embedding) AS embedding_similarity,
         ($embedding_weight * gds.similarity.cosine(e.embedding, $query_embedding) + $structure_weight * structural_score) AS hybrid_score
    
    WHERE hybrid_score >= $similarity_threshold
    
    // Late text collection for final candidates
    CALL {
        WITH e
        OPTIONAL MATCH (e)-[:RESULTS_IN]->(effect:Effect)
        RETURN COLLECT(effect.text) AS effect_texts
    }
    CALL {
        WITH e
        OPTIONAL MATCH (e)<-[:CAUSES]-(cause:Cause)
        RETURN COLLECT(cause.text) AS cause_texts
    }
    CALL {
        WITH e
        OPTIONAL MATCH (e)-[:HAS_TRIGGER]->(trigger:Trigger)
        RETURN COLLECT(trigger.text) AS trigger_texts
    }
    
    RETURN e.text AS text,
           e.id AS event_id,
           hybrid_score,
           embedding_similarity,
           structural_score,
           effect_count,
           cause_count,
           trigger_count,
           effect_texts,
           cause_texts,
           trigger_texts
    ORDER BY hybrid_score DESC
    LIMIT $top_k
    """
    
    try:
        logging.getLogger("neo4j").setLevel(logging.ERROR)
        results = graph.query(
            cypher_query,
            {
                "query_embedding": query_embedding,
                "embedding_weight": float(embedding_weight),
                "structure_weight": float(structure_weight),
                "similarity_threshold": float(similarity_threshold),
                "top_k": int(top_k)
            }
        )
        
        similar_events = []
        for record in results:
            if record['text']:
                similar_events.append({
                    'text': record['text'],
                    'event_id': record.get('event_id', 'N/A'),
                    'hybrid_score': round(float(record['hybrid_score']), 3),
                    'embedding_similarity': round(float(record['embedding_similarity']), 3),
                    'structural_score': round(float(record['structural_score']), 3),
                    'connection_counts': {
                        'effects': int(record['effect_count']),
                        'causes': int(record['cause_count']),
                        'triggers': int(record['trigger_count'])
                    },
                    'connected_nodes': {
                        'causes': [str(cause) for cause in record['cause_texts'] if cause],
                        'effects': [str(effect) for effect in record['effect_texts'] if effect],
                        'triggers': [str(trigger) for trigger in record['trigger_texts'] if trigger]
                    }
                })
        
        return similar_events
    
    except Exception as e:
        print(f"Error during query execution: {str(e)}")
        return []

In [5]:
def generate(llm, input, graph, embeddings, similar_number):
    prompt="""
    <task>
        You are an advanced language model tasked with identifying and tagging explicit cause-and-effect relationships in complex sentences. 
        <rule>Do not modify the original text when adding tags.</rule>
    </task>

    <VERY IMPORTANT RULE>Do not give output unless all tags are present in sentence (All tags meaning cause, effect, and trigger should be present). If you miss this in the end the answer will be considered wrong</VERY IMPORTANT RULE>

    <steps>
        <step number="1">
            <title>Causality Determination</title>
            <description>
                <point>Analyze the input sentence to determine if it contains an explicit causal relationship.</point>
                <point>If the sentence is <b>not causal</b>, output <b>NoTag</b> and set the label to <b>0</b>. <rule>Do not perform any tagging.</rule></point>
                <point>If the sentence is causal, proceed only if <b>all three elements</b>—cause, trigger, and effect—are present.</point>
                <point>If any element is missing, output <b>NoTag</b> and set the label to <b>0</b>.</point>
            </description>
        </step>

        <step number="2">
            <title>Tagging Instructions</title>
            <tags>
                <tag>
                    <name>Cause</name>
                    <format>&lt;cause&gt;...&lt;/cause&gt;</format>
                    <definition>The event, action, or condition that leads to an outcome.</definition>
                    <question>What caused this?</question>
                </tag>
                <tag>
                    <name>Trigger</name>
                    <format>&lt;trigger&gt;...&lt;/trigger&gt;</format>
                    <definition>The word or phrase that explicitly indicates causality (e.g., because, due to).</definition>
                </tag>
                <tag>
                    <name>Effect</name>
                    <format>&lt;effect&gt;...&lt;/effect&gt;</format>
                    <definition>The outcome or result of the cause.</definition>
                    <question>What is the result of this?</question>
                </tag>
            </tags>
        </step>

        <step number="3">
            <title>Causality Tests</title>
            <description>
                <point>The effect should answer "Why?" (i.e., it must clearly result from the cause).</point>
                <point>The cause must precede the effect in time.</point>
                <point>The effect should not occur without the cause.</point>
                <point>The cause and effect cannot be swapped without changing the meaning.</point>
                <point>The sentence can be rephrased as “X causes Y” or “Due to X, Y.”</point>
            </description>
        </step>

        <step number="4">
            <title>Additional Guidelines</title>
            <guidelines>
                <point>The sentence must have explicit cause-effect wording.</point>
                <point>If there are multiple causes or effects, tag each instance separately.</point>
                <point>Do not alter the original sentence structure when adding tags.</point>
                <point>The final output must include all three tags: <b>&lt;cause&gt;</b>, <b>&lt;trigger&gt;</b>, and <b>&lt;effect&gt;</b>.</point>
                <point>If any tag is missing, consider the sentence non-causal.</point>
                <point>If no explicit causal relationship is identified, output <b>NoTag</b>.</point>
            </guidelines>
        </step>

        <step number="5">
            <title>Output Format</title>
            <output>
                <format>Return a JSON object with two keys: "tagged" and "label".</format>
                <key name="tagged">Contains the original sentence with all tags added.</key>
                <key name="label">Set to <b>1</b> if a causal relationship is successfully tagged; otherwise, <b>0</b>.</key>
            </output>
        </step>
    </steps>

    <examples>
        <example>
            <input>Biswal, who had travelled to the city from Washington to speak at the Indian Consulate General's Media-India Lecture Series yesterday, said not only were there a large number of Indian victims in the attack but there were Americans also who lost their lives.</input>
            <tagged>Biswal, who had <trigger>travelled to the city from Washington</trigger> to <cause>speak at the Indian Consulate General's Media-India Lecture Series</cause> yesterday, said <effect>not only were there a large number of Indian victims in the attack</effect> but <effect>there were Americans also who lost their lives</effect>.</tagged>
            <label>1</label>
        </example>

        <more examples>
        {similar_examples}
        </more examples>

        <example>
            <input>The African nationals had sustained minor injuries in the attack that took place in South Delhi's Mehrauli area on Friday.</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>
        
        <example>
            <input>On Sunday , Swaraj spoke to Rajnath Singh and Delhi Lieutenant Governor Najeeb Jung about the attacks and said she was assured that the culprits would soon be arrested .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

        <example>
            <input>We will continue our strike till we get an assurance from the Government .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

        <example>
            <input>There is no information about the involvement of the NDFB(S) underground group in the blast , he added .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

    </examples>

    <qa>
        <requirement>For sentences that are tagged as causal, include a brief description explaining the cause-and-effect relationship.</requirement>
        <example>
            <sentence>&lt;effect>The participants raised slogans&lt;/effect> &lt;trigger>demanding&lt;/trigger> &lt;cause>stern action to check atrocities against women in the name of moral policing&lt;/cause></sentence>
            <explanation>Why did they raise slogans? To demand action against atrocities.</explanation>
            <label>1</label>
        </example>
    </qa>

    <final_instruction>
        Now, process the following input text:
        <input>{input_text}</input>
        <note>Reconsider your decisions step by step internally (chain-of-thought) but do not output your reasoning.</note>
        <IMPORTANT>Do not give output unless all tags are present in sentence (All tags meaning cause, effect, and trigger should be present).</IMPORTANT>
    </final_instruction>
    """
    prompt_template = PromptTemplate(
		input_variables=["similar_examples", "input_text"],
		template=prompt
	)

    llm_chain = LLMChain(
    	llm=llm,
    	prompt=prompt_template
	)

    similar_events = retrieve_similar_events(
        input,
        graph,
        embeddings,
        top_k=similar_number,
        embedding_weight=0.6,
        structure_weight=0.4,
        similarity_threshold=0.5
    )
    
    output = llm_chain.run(
	    similar_examples=similar_events,
	    input_text=input
	)

    return output

In [6]:
llm = ChatGroq(
    groq_api_key=os.environ.get('GROQ_API'),
    model_name='meta-llama/llama-4-maverick-17b-128e-instruct'
)

In [12]:
query_example = "As commuters rushed past , activists waved flags in the sticky night air and erected tall blue and white banners urging them to join the struggle for self-determination ."
llm_output = generate(llm, query_example, graph, embeddings, 5)

print(llm_output)

To address the task, we need to analyze the given input sentence: "As commuters rushed past, activists waved flags in the sticky night air and erected tall blue and white banners urging them to join the struggle for self-determination."

1. **Causality Determination**: The first step is to determine if the sentence contains an explicit causal relationship. Upon examination, the sentence describes an action (activists waving flags and erecting banners) and its context (commuters rushing past), but it doesn't explicitly state a cause-and-effect relationship. The sentence is more about describing a scene and an action taken by activists.

2. **Identifying Cause, Trigger, and Effect**: 
   - **Cause**: The cause is the event or condition that leads to an outcome. In this sentence, "commuters rushed past" could be considered a context or a circumstance rather than a direct cause of an effect.
   - **Trigger**: The trigger is the word or phrase that indicates causality. The sentence lacks a 

In [8]:
df = pd.read_csv('test_data.csv')

In [14]:
df

,index,text,label,agreement,num_votes,sample_set
0,train_10_0,The movement was catapulted into the headlines...,1,0.666667,6,train_10
1,train_10_1,Several thousand protesters took to the street...,1,0.833333,6,train_10
2,train_10_2,"“ They try every means to oppress us , ” compl...",1,0.833333,6,train_10
3,train_10_3,"Chan Ho-tin , the 25-year-old founder of the H...",0,0.833333,6,train_10
4,train_10_4,“ It is somehow like a revolution – no one wil...,1,0.500000,6,train_10
...,...,...,...,...,...,...
318,train_10_318,"Police spokesman , Lieutenant Colonel Thulani ...",0,0.500000,4,train_10
319,train_10_319,"National Freedom Party spokesman , Sabelo Sigu...",1,0.750000,4,train_10
320,train_10_320,He described the shooting as `` evil '' .,0,1.000000,4,train_10
321,train_10_321,"On Sunday , three explosions in the south of t...",1,1.000000,4,train_10


In [15]:
import threading
import time
import json
import re
from tqdm import tqdm

lock = threading.Lock()
sem = threading.Semaphore(5)

if os.path.exists("test_pred_25_maverick.json"):
    with open("test_pred_25_maverick.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []
    with open("test_pred_25_maverick.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

processed = {item["original"] for item in data}

total_rows = len(df)
start_time = time.time()
pbar = tqdm(total=total_rows, desc="Processing", unit="rows")

def worker(text):
    output = generate(llm, text, graph, embeddings, 25)
    match = re.search(r"\{.*\}", output, re.DOTALL)
    if not match:
        with lock:
            pbar.update(1)
        sem.release()
        return

    try:
        j = json.loads(match.group(0))
    except json.JSONDecodeError:
        with lock:
            pbar.update(1)
        sem.release()
        return

    tagged = j.get("tagged", "")
    label = j.get("label", "")

    with lock:
        data.append({"tagged": tagged, "label": label, "original": text})
        with open("test_pred_25_maverick.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        pbar.update(1)
        elapsed = time.time() - start_time
        avg = elapsed / pbar.n
        rem = avg * (total_rows - pbar.n)
        pbar.set_postfix({"ETA (sec)": f"{rem:.2f}"})
        print(f"\nOriginal Sentence: {text}")
        print(f"Tagged Sentence: {tagged}")
        print(f"Label: {label}")
        print("---")
    sem.release()

threads = []
for _, row in df.iterrows():
    text = row["text"]
    if text in processed:
        pbar.update(1)
        continue
    sem.acquire()
    t = threading.Thread(target=worker, args=(text,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

pbar.close()

Processing:   0%|          | 1/323 [00:01<05:36,  1.05s/rows, ETA (sec)=338.48]


Original Sentence: The movement was catapulted into the headlines in early August when the semi-autonomous city – which returned to Chinese rule almost two decades ago , in 1997 – saw the first pro-independence rally in its history .
Tagged Sentence: <effect>The movement was catapulted into the headlines</effect> <trigger>in early August when</trigger> the semi-autonomous city – which <cause>returned to Chinese rule almost two decades ago , in1997 – saw the first pro-independence rally in its history</cause>.
Label: 1
---


Processing:   1%|          | 2/323 [00:01<05:10,  1.03rows/s, ETA (sec)=315.87]


Original Sentence: As commuters rushed past , activists waved flags in the sticky night air and erected tall blue and white banners urging them to join the struggle for self-determination .
Tagged Sentence: As commuters rushed past , <cause>activists waved flags in the sticky night air</cause> <trigger>and</trigger> erected <effect>tall blue and white banners urging them to join the struggle for self-determination</effect> .
Label: 1
---


Processing:   1%|          | 3/323 [00:02<05:07,  1.04rows/s, ETA (sec)=311.12]

...


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing:  99%|█████████▉| 319/323 [05:00<00:04,  1.14s/rows, ETA (sec)=3.77]


Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: On Sunday , <cause>three explosions in the south of the region</cause> <trigger>killed</trigger> <effect>two people</effect> and <trigger>injured</trigger> <effect>many others</effect> .
Label: 1
---


Processing:  99%|█████████▉| 320/323 [05:01<00:02,  1.06rows/s, ETA (sec)=2.82]


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: <effect>the clashes</effect> are a desperate protest <trigger>against</trigger> <cause>religious oppression and economic marginalisation</cause>
Label: 1
---


# Get F1 Score 25

In [16]:
with open("test_pred_25_maverick.json", "r", encoding="utf-8") as file:
    data = json.load(file)

df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.7833827893175074
Accuracy: 0.7213740458015268
Precision: 0.6839378238341969
Recall: 0.9166666666666666
Matthews Correlation Coefficient: 0.45151504664370123


# 30 Examples

In [10]:
import threading
import time
import json
import re
from tqdm import tqdm

lock = threading.Lock()
sem = threading.Semaphore(5)

if os.path.exists("test_pred_30_maverick.json"):
    with open("test_pred_30_maverick.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []
    with open("test_pred_30_maverick.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

processed = {item["original"] for item in data}

total_rows = len(df)
start_time = time.time()
pbar = tqdm(total=total_rows, desc="Processing", unit="rows")

def worker(text):
    output = generate(llm, text, graph, embeddings, 30)
    match = re.search(r"\{.*\}", output, re.DOTALL)
    if not match:
        with lock:
            pbar.update(1)
        sem.release()
        return

    try:
        j = json.loads(match.group(0))
    except json.JSONDecodeError:
        with lock:
            pbar.update(1)
        sem.release()
        return

    tagged = j.get("tagged", "")
    label = j.get("label", "")

    with lock:
        data.append({"tagged": tagged, "label": label, "original": text})
        with open("test_pred_30_maverick.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        pbar.update(1)
        elapsed = time.time() - start_time
        avg = elapsed / pbar.n
        rem = avg * (total_rows - pbar.n)
        pbar.set_postfix({"ETA (sec)": f"{rem:.2f}"})
        print(f"\nOriginal Sentence: {text}")
        print(f"Tagged Sentence: {tagged}")
        print(f"Label: {label}")
        print("---")
    sem.release()

threads = []
for _, row in df.iterrows():
    text = row["text"]
    if text in processed:
        pbar.update(1)
        continue
    sem.acquire()
    t = threading.Thread(target=worker, args=(text,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

pbar.close()

Processing:   0%|          | 1/323 [00:01<07:41,  1.43s/rows, ETA (sec)=466.20]


Original Sentence: Several thousand protesters took to the streets after six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing ’ s authoritarian rule .
Tagged Sentence: <effect>Several thousand protesters took to the streets</effect> <trigger>after</trigger> <cause>six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election</cause>
Label: 1
---


Processing:   1%|          | 2/323 [00:02<06:44,  1.26s/rows, ETA (sec)=413.43]


Original Sentence: As commuters rushed past , activists waved flags in the sticky night air and erected tall blue and white banners urging them to join the struggle for self-determination .
Tagged Sentence: As commuters rushed past , <cause>activists waved flags in the sticky night air and erected tall blue and white banners</cause> <trigger>urging</trigger> <effect>them to join the struggle for self-determination</effect> .
Label: 1
---


Processing:   1%|          | 3/323 [00:03<06:21,  1.19s/rows, ETA (sec)=393.76]

...


Original Sentence: National Freedom Party spokesman , Sabelo Sigudu , said : `` The National Freedom Party ( NFP ) wishes to express words of condolence on the sad shooting and killing of Mr Sithole , the Municipality Manager of Richmond Local Municipality , which took place this morning . ''
Tagged Sentence: NoTag
Label: 0
---


Processing:  98%|█████████▊| 317/323 [05:53<00:05,  1.08rows/s, ETA (sec)=6.70]


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing:  99%|█████████▉| 319/323 [05:59<00:04,  1.13s/rows, ETA (sec)=4.51]


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: <effect>the clashes</effect> are a desperate protest <trigger>against</trigger> <cause>religious oppression and economic marginalisation</cause>
Label: 1
---


# Get F1 Score

In [11]:
with open("test_pred_30_maverick.json", "r", encoding="utf-8") as file:
    data = json.load(file)

df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.8108108108108109
Accuracy: 0.7519685039370079
Precision: 0.7219251336898396
Recall: 0.9246575342465754
Matthews Correlation Coefficient: 0.49717168430199216


# 40 Examples

In [13]:
import threading
import time
import json
import re
from tqdm import tqdm

lock = threading.Lock()
sem = threading.Semaphore(5)

if os.path.exists("test_pred_40_maverick.json"):
    with open("test_pred_40_maverick.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []
    with open("test_pred_40_maverick.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

processed = {item["original"] for item in data}

total_rows = len(df)
start_time = time.time()
pbar = tqdm(total=total_rows, desc="Processing", unit="rows")

def worker(text):
    output = generate(llm, text, graph, embeddings, 40)
    match = re.search(r"\{.*\}", output, re.DOTALL)
    if not match:
        with lock:
            pbar.update(1)
        sem.release()
        return

    try:
        j = json.loads(match.group(0))
    except json.JSONDecodeError:
        with lock:
            pbar.update(1)
        sem.release()
        return

    tagged = j.get("tagged", "")
    label = j.get("label", "")

    with lock:
        data.append({"tagged": tagged, "label": label, "original": text})
        with open("test_pred_40_maverick.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        pbar.update(1)
        elapsed = time.time() - start_time
        avg = elapsed / pbar.n
        rem = avg * (total_rows - pbar.n)
        pbar.set_postfix({"ETA (sec)": f"{rem:.2f}"})
        print(f"\nOriginal Sentence: {text}")
        print(f"Tagged Sentence: {tagged}")
        print(f"Label: {label}")
        print("---")
    sem.release()

threads = []
for _, row in df.iterrows():
    text = row["text"]
    if text in processed:
        pbar.update(1)
        continue
    sem.acquire()
    t = threading.Thread(target=worker, args=(text,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

pbar.close()

Processing:   0%|          | 1/323 [00:01<08:29,  1.58s/rows, ETA (sec)=510.95]


Original Sentence: Several thousand protesters took to the streets after six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing ’ s authoritarian rule .
Tagged Sentence: <effect>Several thousand protesters took to the streets</effect> <trigger>after</trigger> <cause>six pro-independence candidates, foremost amongst them Leung, were barred from the upcoming election</cause>
Label: 1
---


Processing:   1%|          | 2/323 [00:01<04:24,  1.21rows/s, ETA (sec)=303.78]


Original Sentence: Chan Ho-tin , the 25-year-old founder of the Hong Kong national party , one of several recently formed pro-independence groups , said he had been surprised how many people attended August ’ s historic rally , which he organised .
Tagged Sentence: <effect>he had been surprised</effect> how many people attended August 's historic rally <cause>the historic rally</cause> <trigger>which he organised</trigger></effect>
Label: 1
---


Processing:   1%|          | 3/323 [00:02<03:01,  1.76rows/s, ETA (sec)=228.72]

...


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing:  98%|█████████▊| 318/323 [07:42<00:05,  1.11s/rows, ETA (sec)=7.28]


Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: On Sunday , <cause>three explosions in the south of the region</cause> <trigger>killed</trigger> <effect>two people</effect> and <trigger>injured</trigger> <effect>many others</effect> .
Label: 1
---


Processing:  99%|█████████▉| 319/323 [07:43<00:05,  1.45s/rows, ETA (sec)=5.82]


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: <effect>the clashes</effect> are <trigger>a desperate protest against</trigger> <cause>religious oppression and economic marginalisation</cause>
Label: 1
---


# Get F1 Score

In [14]:
with open("test_pred_40_maverick.json", "r", encoding="utf-8") as file:
    data = json.load(file)

df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.7950310559006211
Accuracy: 0.7327935222672065
Precision: 0.6994535519125683
Recall: 0.920863309352518
Matthews Correlation Coefficient: 0.4659972840152072


# 50 Examples

In [15]:
import threading
import time
import json
import re
from tqdm import tqdm

lock = threading.Lock()
sem = threading.Semaphore(5)

if os.path.exists("test_pred_50_maverick.json"):
    with open("test_pred_50_maverick.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []
    with open("test_pred_50_maverick.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

processed = {item["original"] for item in data}

total_rows = len(df)
start_time = time.time()
pbar = tqdm(total=total_rows, desc="Processing", unit="rows")

def worker(text):
    output = generate(llm, text, graph, embeddings, 50)
    match = re.search(r"\{.*\}", output, re.DOTALL)
    if not match:
        with lock:
            pbar.update(1)
        sem.release()
        return

    try:
        j = json.loads(match.group(0))
    except json.JSONDecodeError:
        with lock:
            pbar.update(1)
        sem.release()
        return

    tagged = j.get("tagged", "")
    label = j.get("label", "")

    with lock:
        data.append({"tagged": tagged, "label": label, "original": text})
        with open("test_pred_50_maverick.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        pbar.update(1)
        elapsed = time.time() - start_time
        avg = elapsed / pbar.n
        rem = avg * (total_rows - pbar.n)
        pbar.set_postfix({"ETA (sec)": f"{rem:.2f}"})
        print(f"\nOriginal Sentence: {text}")
        print(f"Tagged Sentence: {tagged}")
        print(f"Label: {label}")
        print("---")
    sem.release()

threads = []
for _, row in df.iterrows():
    text = row["text"]
    if text in processed:
        pbar.update(1)
        continue
    sem.acquire()
    t = threading.Thread(target=worker, args=(text,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

pbar.close()

Processing:   0%|          | 1/323 [00:01<07:09,  1.33s/rows, ETA (sec)=430.39]


Original Sentence: Several thousand protesters took to the streets after six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing ’ s authoritarian rule .
Tagged Sentence: Several thousand protesters took to the streets <trigger>after</trigger> <cause>six pro-independence candidates, foremost amongst them Leung, were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing’s authoritarian rule</cause> .
Label: 1
---


Processing:   1%|          | 2/323 [00:01<03:39,  1.46rows/s, ETA (sec)=251.94]


Original Sentence: Chan Ho-tin , the 25-year-old founder of the Hong Kong national party , one of several recently formed pro-independence groups , said he had been surprised how many people attended August ’ s historic rally , which he organised .
Tagged Sentence: NoTag
Label: 0
---


Processing:   2%|▏         | 5/323 [00:02<01:46,  2.99rows/s, ETA (sec)=151.72]

...


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing:  98%|█████████▊| 318/323 [08:35<00:13,  2.61s/rows, ETA (sec)=8.11]


Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: On Sunday , <cause>three explosions in the south of the region</cause> killed <effect>two people and injured many others</effect> .
Label: 1
---


Processing:  99%|█████████▉| 319/323 [08:36<00:06,  1.62s/rows, ETA (sec)=6.48]


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: Uighur groups abroad say the clashes are a desperate <effect>protest</effect> <trigger>against</trigger> <cause>religious oppression and economic marginalisation</cause> .
Label: 1
---


# F1 Score

In [16]:
with open("test_pred_50_maverick.json", "r", encoding="utf-8") as file:
    data = json.load(file)

df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.7851002865329513
Accuracy: 0.7137404580152672
Precision: 0.6748768472906403
Recall: 0.9383561643835616
Matthews Correlation Coefficient: 0.43925558565226835


# 30 Examples Iteration 2

In [17]:
import threading
import time
import json
import re
from tqdm import tqdm

lock = threading.Lock()
sem = threading.Semaphore(5)

if os.path.exists("test_pred_30_maverick_iter_2.json"):
    with open("test_pred_30_maverick_iter_2.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []
    with open("test_pred_30_maverick_iter_2.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

processed = {item["original"] for item in data}

total_rows = len(df)
start_time = time.time()
pbar = tqdm(total=total_rows, desc="Processing", unit="rows")

def worker(text):
    output = generate(llm, text, graph, embeddings, 30)
    match = re.search(r"\{.*\}", output, re.DOTALL)
    if not match:
        with lock:
            pbar.update(1)
        sem.release()
        return

    try:
        j = json.loads(match.group(0))
    except json.JSONDecodeError:
        with lock:
            pbar.update(1)
        sem.release()
        return

    tagged = j.get("tagged", "")
    label = j.get("label", "")

    with lock:
        data.append({"tagged": tagged, "label": label, "original": text})
        with open("test_pred_30_maverick_iter_2.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        pbar.update(1)
        elapsed = time.time() - start_time
        avg = elapsed / pbar.n
        rem = avg * (total_rows - pbar.n)
        pbar.set_postfix({"ETA (sec)": f"{rem:.2f}"})
        print(f"\nOriginal Sentence: {text}")
        print(f"Tagged Sentence: {tagged}")
        print(f"Label: {label}")
        print("---")
    sem.release()

threads = []
for _, row in df.iterrows():
    text = row["text"]
    if text in processed:
        pbar.update(1)
        continue
    sem.acquire()
    t = threading.Thread(target=worker, args=(text,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

pbar.close()

Processing:   1%|          | 2/323 [00:01<05:58,  1.12s/rows, ETA (sec)=180.77]


Original Sentence: The movement was catapulted into the headlines in early August when the semi-autonomous city – which returned to Chinese rule almost two decades ago , in 1997 – saw the first pro-independence rally in its history .
Tagged Sentence: <effect>The movement was catapulted into the headlines</effect> <trigger>in early August when</trigger> <cause>the semi-autonomous city saw the first pro-independence rally in its history</cause>
Label: 1
---

Original Sentence: “ It is somehow like a revolution – no one will have a plan for a revolution. ” On a recent evening , Edward Leung and a group of fellow activists gathered outside a metro station in Hong Kong ’ s New Territories to lend their support to a Youngspiration campaign event .
Tagged Sentence: NoTag
Label: 0
---


Processing:   1%|          | 3/323 [00:01<02:27,  2.17rows/s, ETA (sec)=169.02]


Original Sentence: Several thousand protesters took to the streets after six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing ’ s authoritarian rule .
Tagged Sentence: <effect>Several thousand protesters took to the streets</effect> <trigger>after</trigger> <cause>six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election</cause>
Label: 1
---


Processing:   1%|          | 4/323 [00:01<02:09,  2.47rows/s, ETA (sec)=150.37]

...


Original Sentence: National Freedom Party spokesman , Sabelo Sigudu , said : `` The National Freedom Party ( NFP ) wishes to express words of condolence on the sad shooting and killing of Mr Sithole , the Municipality Manager of Richmond Local Municipality , which took place this morning . ''
Tagged Sentence: NoTag
Label: 0
---


Processing:  98%|█████████▊| 317/323 [07:18<00:08,  1.48s/rows, ETA (sec)=8.31]


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing:  99%|█████████▉| 319/323 [07:23<00:05,  1.39s/rows, ETA (sec)=5.56]


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: <effect>the clashes</effect> are <trigger>a desperate protest against</trigger> <cause>religious oppression and economic marginalisation</cause>
Label: 1
---


In [19]:
with open("test_pred_30_maverick_iter_2.json", "r", encoding="utf-8") as file:
    data = json.load(file)

df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.8152492668621701
Accuracy: 0.7567567567567568
Precision: 0.7202072538860104
Recall: 0.9391891891891891
Matthews Correlation Coefficient: 0.5141085329073053


# 30 Examples iter 3 Changed Hyper parameters, all thresholds above 80%

In [7]:
def generate(llm, input, graph, embeddings, similar_number):
    prompt="""
    <task>
        You are an advanced language model tasked with identifying and tagging explicit cause-and-effect relationships in complex sentences. 
        <rule>Do not modify the original text when adding tags.</rule>
    </task>

    <VERY IMPORTANT RULE>Do not give output unless all tags are present in sentence (All tags meaning cause, effect, and trigger should be present). If you miss this in the end the answer will be considered wrong</VERY IMPORTANT RULE>

    <steps>
        <step number="1">
            <title>Causality Determination</title>
            <description>
                <point>Analyze the input sentence to determine if it contains an explicit causal relationship.</point>
                <point>If the sentence is <b>not causal</b>, output <b>NoTag</b> and set the label to <b>0</b>. <rule>Do not perform any tagging.</rule></point>
                <point>If the sentence is causal, proceed only if <b>all three elements</b>—cause, trigger, and effect—are present.</point>
                <point>If any element is missing, output <b>NoTag</b> and set the label to <b>0</b>.</point>
            </description>
        </step>

        <step number="2">
            <title>Tagging Instructions</title>
            <tags>
                <tag>
                    <name>Cause</name>
                    <format>&lt;cause&gt;...&lt;/cause&gt;</format>
                    <definition>The event, action, or condition that leads to an outcome.</definition>
                    <question>What caused this?</question>
                </tag>
                <tag>
                    <name>Trigger</name>
                    <format>&lt;trigger&gt;...&lt;/trigger&gt;</format>
                    <definition>The word or phrase that explicitly indicates causality (e.g., because, due to).</definition>
                </tag>
                <tag>
                    <name>Effect</name>
                    <format>&lt;effect&gt;...&lt;/effect&gt;</format>
                    <definition>The outcome or result of the cause.</definition>
                    <question>What is the result of this?</question>
                </tag>
            </tags>
        </step>

        <step number="3">
            <title>Causality Tests</title>
            <description>
                <point>The effect should answer "Why?" (i.e., it must clearly result from the cause).</point>
                <point>The cause must precede the effect in time.</point>
                <point>The effect should not occur without the cause.</point>
                <point>The cause and effect cannot be swapped without changing the meaning.</point>
                <point>The sentence can be rephrased as “X causes Y” or “Due to X, Y.”</point>
            </description>
        </step>

        <step number="4">
            <title>Additional Guidelines</title>
            <guidelines>
                <point>The sentence must have explicit cause-effect wording.</point>
                <point>If there are multiple causes or effects, tag each instance separately.</point>
                <point>Do not alter the original sentence structure when adding tags.</point>
                <point>The final output must include all three tags: <b>&lt;cause&gt;</b>, <b>&lt;trigger&gt;</b>, and <b>&lt;effect&gt;</b>.</point>
                <point>If any tag is missing, consider the sentence non-causal.</point>
                <point>If no explicit causal relationship is identified, output <b>NoTag</b>.</point>
            </guidelines>
        </step>

        <step number="5">
            <title>Output Format</title>
            <output>
                <format>Return a JSON object with two keys: "tagged" and "label".</format>
                <key name="tagged">Contains the original sentence with all tags added.</key>
                <key name="label">Set to <b>1</b> if a causal relationship is successfully tagged; otherwise, <b>0</b>.</key>
            </output>
        </step>
    </steps>

    <examples>
        <example>
            <input>Biswal, who had travelled to the city from Washington to speak at the Indian Consulate General's Media-India Lecture Series yesterday, said not only were there a large number of Indian victims in the attack but there were Americans also who lost their lives.</input>
            <tagged>Biswal, who had <trigger>travelled to the city from Washington</trigger> to <cause>speak at the Indian Consulate General's Media-India Lecture Series</cause> yesterday, said <effect>not only were there a large number of Indian victims in the attack</effect> but <effect>there were Americans also who lost their lives</effect>.</tagged>
            <label>1</label>
        </example>

        <more examples>
        {similar_examples}
        </more examples>

        <example>
            <input>The African nationals had sustained minor injuries in the attack that took place in South Delhi's Mehrauli area on Friday.</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>
        
        <example>
            <input>On Sunday , Swaraj spoke to Rajnath Singh and Delhi Lieutenant Governor Najeeb Jung about the attacks and said she was assured that the culprits would soon be arrested .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

        <example>
            <input>We will continue our strike till we get an assurance from the Government .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

        <example>
            <input>There is no information about the involvement of the NDFB(S) underground group in the blast , he added .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

    </examples>

    <qa>
        <requirement>For sentences that are tagged as causal, include a brief description explaining the cause-and-effect relationship.</requirement>
        <example>
            <sentence>&lt;effect>The participants raised slogans&lt;/effect> &lt;trigger>demanding&lt;/trigger> &lt;cause>stern action to check atrocities against women in the name of moral policing&lt;/cause></sentence>
            <explanation>Why did they raise slogans? To demand action against atrocities.</explanation>
            <label>1</label>
        </example>
    </qa>

    <final_instruction>
        Now, process the following input text:
        <input>{input_text}</input>
        <note>Reconsider your decisions step by step internally (chain-of-thought) but do not output your reasoning.</note>
        <IMPORTANT>Do not give output unless all tags are present in sentence (All tags meaning cause, effect, and trigger should be present).</IMPORTANT>
    </final_instruction>
    """
    prompt_template = PromptTemplate(
		input_variables=["similar_examples", "input_text"],
		template=prompt
	)

    llm_chain = LLMChain(
    	llm=llm,
    	prompt=prompt_template
	)

    similar_events = retrieve_similar_events(
        input,
        graph,
        embeddings,
        top_k=similar_number,
        embedding_weight=0.8,
        structure_weight=0.8,
        similarity_threshold=0.8
    )
    
    output = llm_chain.run(
	    similar_examples=similar_events,
	    input_text=input
	)

    return output

In [ ]:
import threading
import time
import json
import re
from tqdm import tqdm

lock = threading.Lock()
sem = threading.Semaphore(5)

if os.path.exists("test_pred_30_maverick_iter_3.json"):
    with open("test_pred_30_maverick_iter_3.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []
    with open("test_pred_30_maverick_iter_3.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

processed = {item["original"] for item in data}

total_rows = len(df)
start_time = time.time()
pbar = tqdm(total=total_rows, desc="Processing", unit="rows")

def worker(text):
    output = generate(llm, text, graph, embeddings, 30)
    match = re.search(r"\{.*\}", output, re.DOTALL)
    if not match:
        with lock:
            pbar.update(1)
        sem.release()
        return

    try:
        j = json.loads(match.group(0))
    except json.JSONDecodeError:
        with lock:
            pbar.update(1)
        sem.release()
        return

    tagged = j.get("tagged", "")
    label = j.get("label", "")

    with lock:
        data.append({"tagged": tagged, "label": label, "original": text})
        with open("test_pred_30_maverick_iter_3.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        pbar.update(1)
        elapsed = time.time() - start_time
        avg = elapsed / pbar.n
        rem = avg * (total_rows - pbar.n)
        pbar.set_postfix({"ETA (sec)": f"{rem:.2f}"})
        print(f"\nOriginal Sentence: {text}")
        print(f"Tagged Sentence: {tagged}")
        print(f"Label: {label}")
        print("---")
    sem.release()

threads = []
for _, row in df.iterrows():
    text = row["text"]
    if text in processed:
        pbar.update(1)
        continue
    sem.acquire()
    t = threading.Thread(target=worker, args=(text,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

pbar.close()

Processing:   0%|          | 1/323 [00:01<06:48,  1.27s/rows, ETA (sec)=410.12]


Original Sentence: “ It is somehow like a revolution – no one will have a plan for a revolution. ” On a recent evening , Edward Leung and a group of fellow activists gathered outside a metro station in Hong Kong ’ s New Territories to lend their support to a Youngspiration campaign event .
Tagged Sentence: NoTag
Label: 0
---


Processing:   1%|          | 3/323 [00:01<02:23,  2.24rows/s, ETA (sec)=183.78]


Original Sentence: The movement was catapulted into the headlines in early August when the semi-autonomous city – which returned to Chinese rule almost two decades ago , in 1997 – saw the first pro-independence rally in its history .
Tagged Sentence: <effect>The movement was catapulted into the headlines</effect> <trigger>in early August when</trigger> <cause>the semi-autonomous city saw the first pro-independence rally in its history</cause>
Label: 1
---

Original Sentence: Several thousand protesters took to the streets after six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing ’ s authoritarian rule .
Tagged Sentence: <effect>Several thousand protesters took to the streets</effect> <trigger>after</trigger> <cause>six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election</cause>
Label: 1

Processing:   1%|          | 4/323 [00:02<02:27,  2.16rows/s, ETA (sec)=176.40]

...

Processing:  80%|███████▉  | 257/323 [03:36<01:32,  1.41s/rows, ETA (sec)=55.70]


Original Sentence: Raising slogans against the civic body , the protesters , led by area councillor Kamlesh , demanded that the waste be taken to the garbage processing plant that has been set up to process the refuse .
Tagged Sentence: <effect>Raising slogans against the civic body</effect> , the protesters , led by area councillor Kamlesh , <trigger>demanded</trigger> that <cause>the waste be taken to the garbage processing plant that has been set up to process the refuse</cause>.
Label: 1
---


Processing:  80%|███████▉  | 258/323 [03:37<01:22,  1.27s/rows, ETA (sec)=54.86]


Original Sentence: The trucks were not allowed to move till the Municipal Commissioner Prerna Puri reached the site .
Tagged Sentence: <effect>The trucks were not allowed to move</effect> <trigger>till</trigger> <cause>the Municipal Commissioner Prerna Puri reached the site</cause>.
Label: 1
---


Exception in thread Thread-1879 (worker):
Traceback (most recent call last):
  File "C:\Users\ahaqu\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1009, in _bootstrap_inner
    self.run()
  File "c:\Users\ahaqu\OneDrive\Desktop\FYP\graphRAG\grag\lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "C:\Users\ahaqu\AppData\Local\Programs\Python\Python310\lib\threading.py", line 946, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\ahaqu\AppData\Local\Temp\ipykernel_14932\2106284612.py", line 25, in worker
  File "C:\Users\ahaqu\AppData\Local\Temp\ipykernel_14932\157567790.py", line 150, in generate
  File "c:\Users\ahaqu\OneDrive\Desktop\FYP\graphRAG\grag\lib\site-packages\langchain_core\_api\deprecation.py", line 182, in warning_emitting_wrapper
    return wrapped(*args, **kwargs)
  File "c:\Users\ahaqu\OneDrive\Desktop\FYP\graphRAG\grag\lib\site-packages\langchain\chains\base.py", line 611, in ru

In [9]:
with open("test_pred_30.json", "r", encoding="utf-8") as file:
    data = json.load(file)

df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.8119891008174387
Accuracy: 0.7816455696202531
Precision: 0.772020725388601
Recall: 0.8563218390804598
Matthews Correlation Coefficient: 0.5575041404023712


# 30 Examples Iteration 4 change Hyperparameters for Hybrid Search: 30% Allowed

In [10]:
def generate(llm, input, graph, embeddings, similar_number):
    prompt="""
    <task>
        You are an advanced language model tasked with identifying and tagging explicit cause-and-effect relationships in complex sentences. 
        <rule>Do not modify the original text when adding tags.</rule>
    </task>

    <VERY IMPORTANT RULE>Do not give output unless all tags are present in sentence (All tags meaning cause, effect, and trigger should be present). If you miss this in the end the answer will be considered wrong</VERY IMPORTANT RULE>

    <steps>
        <step number="1">
            <title>Causality Determination</title>
            <description>
                <point>Analyze the input sentence to determine if it contains an explicit causal relationship.</point>
                <point>If the sentence is <b>not causal</b>, output <b>NoTag</b> and set the label to <b>0</b>. <rule>Do not perform any tagging.</rule></point>
                <point>If the sentence is causal, proceed only if <b>all three elements</b>—cause, trigger, and effect—are present.</point>
                <point>If any element is missing, output <b>NoTag</b> and set the label to <b>0</b>.</point>
            </description>
        </step>

        <step number="2">
            <title>Tagging Instructions</title>
            <tags>
                <tag>
                    <name>Cause</name>
                    <format>&lt;cause&gt;...&lt;/cause&gt;</format>
                    <definition>The event, action, or condition that leads to an outcome.</definition>
                    <question>What caused this?</question>
                </tag>
                <tag>
                    <name>Trigger</name>
                    <format>&lt;trigger&gt;...&lt;/trigger&gt;</format>
                    <definition>The word or phrase that explicitly indicates causality (e.g., because, due to).</definition>
                </tag>
                <tag>
                    <name>Effect</name>
                    <format>&lt;effect&gt;...&lt;/effect&gt;</format>
                    <definition>The outcome or result of the cause.</definition>
                    <question>What is the result of this?</question>
                </tag>
            </tags>
        </step>

        <step number="3">
            <title>Causality Tests</title>
            <description>
                <point>The effect should answer "Why?" (i.e., it must clearly result from the cause).</point>
                <point>The cause must precede the effect in time.</point>
                <point>The effect should not occur without the cause.</point>
                <point>The cause and effect cannot be swapped without changing the meaning.</point>
                <point>The sentence can be rephrased as “X causes Y” or “Due to X, Y.”</point>
            </description>
        </step>

        <step number="4">
            <title>Additional Guidelines</title>
            <guidelines>
                <point>The sentence must have explicit cause-effect wording.</point>
                <point>If there are multiple causes or effects, tag each instance separately.</point>
                <point>Do not alter the original sentence structure when adding tags.</point>
                <point>The final output must include all three tags: <b>&lt;cause&gt;</b>, <b>&lt;trigger&gt;</b>, and <b>&lt;effect&gt;</b>.</point>
                <point>If any tag is missing, consider the sentence non-causal.</point>
                <point>If no explicit causal relationship is identified, output <b>NoTag</b>.</point>
            </guidelines>
        </step>

        <step number="5">
            <title>Output Format</title>
            <output>
                <format>Return a JSON object with two keys: "tagged" and "label".</format>
                <key name="tagged">Contains the original sentence with all tags added.</key>
                <key name="label">Set to <b>1</b> if a causal relationship is successfully tagged; otherwise, <b>0</b>.</key>
            </output>
        </step>
    </steps>

    <examples>
        <example>
            <input>Biswal, who had travelled to the city from Washington to speak at the Indian Consulate General's Media-India Lecture Series yesterday, said not only were there a large number of Indian victims in the attack but there were Americans also who lost their lives.</input>
            <tagged>Biswal, who had <trigger>travelled to the city from Washington</trigger> to <cause>speak at the Indian Consulate General's Media-India Lecture Series</cause> yesterday, said <effect>not only were there a large number of Indian victims in the attack</effect> but <effect>there were Americans also who lost their lives</effect>.</tagged>
            <label>1</label>
        </example>

        <more examples>
        {similar_examples}
        </more examples>

        <example>
            <input>The African nationals had sustained minor injuries in the attack that took place in South Delhi's Mehrauli area on Friday.</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>
        
        <example>
            <input>On Sunday , Swaraj spoke to Rajnath Singh and Delhi Lieutenant Governor Najeeb Jung about the attacks and said she was assured that the culprits would soon be arrested .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

        <example>
            <input>We will continue our strike till we get an assurance from the Government .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

        <example>
            <input>There is no information about the involvement of the NDFB(S) underground group in the blast , he added .</input>
            <tagged>NoTag</tagged>
            <label>0</label>
        </example>

    </examples>

    <qa>
        <requirement>For sentences that are tagged as causal, include a brief description explaining the cause-and-effect relationship.</requirement>
        <example>
            <sentence>&lt;effect>The participants raised slogans&lt;/effect> &lt;trigger>demanding&lt;/trigger> &lt;cause>stern action to check atrocities against women in the name of moral policing&lt;/cause></sentence>
            <explanation>Why did they raise slogans? To demand action against atrocities.</explanation>
            <label>1</label>
        </example>
    </qa>

    <final_instruction>
        Now, process the following input text:
        <input>{input_text}</input>
        <note>Reconsider your decisions step by step internally (chain-of-thought) but do not output your reasoning.</note>
        <IMPORTANT>Do not give output unless all tags are present in sentence (All tags meaning cause, effect, and trigger should be present).</IMPORTANT>
    </final_instruction>
    """
    prompt_template = PromptTemplate(
		input_variables=["similar_examples", "input_text"],
		template=prompt
	)

    llm_chain = LLMChain(
    	llm=llm,
    	prompt=prompt_template
	)

    similar_events = retrieve_similar_events(
        input,
        graph,
        embeddings,
        top_k=similar_number,
        embedding_weight=0.3,
        structure_weight=0.3,
        similarity_threshold=0.3
    )
    
    output = llm_chain.run(
	    similar_examples=similar_events,
	    input_text=input
	)

    return output

In [11]:
import threading
import time
import json
import re
from tqdm import tqdm

lock = threading.Lock()
sem = threading.Semaphore(5)

if os.path.exists("test_pred_30_maverick_iter_4.json"):
    with open("test_pred_30_maverick_iter_4.json", "r", encoding="utf-8") as f:
        data = json.load(f)
else:
    data = []
    with open("test_pred_30_maverick_iter_4.json", "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

processed = {item["original"] for item in data}

total_rows = len(df)
start_time = time.time()
pbar = tqdm(total=total_rows, desc="Processing", unit="rows")

def worker(text):
    output = generate(llm, text, graph, embeddings, 30)
    match = re.search(r"\{.*\}", output, re.DOTALL)
    if not match:
        with lock:
            pbar.update(1)
        sem.release()
        return

    try:
        j = json.loads(match.group(0))
    except json.JSONDecodeError:
        with lock:
            pbar.update(1)
        sem.release()
        return

    tagged = j.get("tagged", "")
    label = j.get("label", "")

    with lock:
        data.append({"tagged": tagged, "label": label, "original": text})
        with open("test_pred_30_maverick_iter_4.json", "w", encoding="utf-8") as f:
            json.dump(data, f, indent=4, ensure_ascii=False)
        pbar.update(1)
        elapsed = time.time() - start_time
        avg = elapsed / pbar.n
        rem = avg * (total_rows - pbar.n)
        pbar.set_postfix({"ETA (sec)": f"{rem:.2f}"})
        print(f"\nOriginal Sentence: {text}")
        print(f"Tagged Sentence: {tagged}")
        print(f"Label: {label}")
        print("---")
    sem.release()

threads = []
for _, row in df.iterrows():
    text = row["text"]
    if text in processed:
        pbar.update(1)
        continue
    sem.acquire()
    t = threading.Thread(target=worker, args=(text,))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

pbar.close()

Processing:   1%|          | 2/323 [00:03<07:54,  1.48s/rows, ETA (sec)=549.98]


Original Sentence: “ It is somehow like a revolution – no one will have a plan for a revolution. ” On a recent evening , Edward Leung and a group of fellow activists gathered outside a metro station in Hong Kong ’ s New Territories to lend their support to a Youngspiration campaign event .
Tagged Sentence: NoTag
Label: 0
---


Processing:   1%|          | 3/323 [00:03<05:35,  1.05s/rows, ETA (sec)=422.75]


Original Sentence: Several thousand protesters took to the streets after six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election in what critics condemned as an act of political censorship designed to snuff out opposition to Beijing ’ s authoritarian rule .
Tagged Sentence: <effect>Several thousand protesters took to the streets</effect> <trigger>after</trigger> <cause>six pro-independence candidates , foremost amongst them Leung , were barred from the upcoming election</cause>
Label: 1
---


Processing:   2%|▏         | 5/323 [00:04<04:44,  1.12rows/s, ETA (sec)=294.65]

...


Original Sentence: He described the shooting as `` evil '' .
Tagged Sentence: NoTag
Label: 0
---


Processing:  98%|█████████▊| 318/323 [06:58<00:07,  1.51s/rows, ETA (sec)=6.57]


Original Sentence: On Sunday , three explosions in the south of the region killed two people and injured many others .
Tagged Sentence: <cause>three explosions in the south of the region</cause> <trigger>killed</trigger> <effect>two people</effect> and injured many others.
Label: 1
---


Processing:  99%|█████████▉| 319/323 [06:59<00:05,  1.31s/rows, ETA (sec)=5.26]


Original Sentence: Uighur groups abroad say the clashes are a desperate protest against religious oppression and economic marginalisation .
Tagged Sentence: <effect>the clashes</effect> <trigger>are a desperate protest against</trigger> <cause>religious oppression and economic marginalisation</cause>
Label: 1
---


In [12]:
with open("test_pred_30_maverick_iter_4.json", "r", encoding="utf-8") as file:
    data = json.load(file)

df_predictions = pd.DataFrame(data, columns=[ "tagged", "label", "original"])

df_predictions.rename(columns={
    "original": "original_sentence",
    "tagged": "tagged_sentence",
    "label": "label"
}, inplace=True)

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce')

df_predictions["label"] = pd.to_numeric(df_predictions["label"], errors='coerce').fillna(0).astype(int)

common_df = df.merge(df_predictions, left_on="text", right_on="original_sentence", suffixes=('_df', '_pred'))

f1 = f1_score(common_df['label_df'], common_df['label_pred'])
print("F1 Score:", f1)

accuracy = accuracy_score(common_df['label_df'], common_df['label_pred'])
print("Accuracy:", accuracy)

precision = precision_score(common_df['label_df'], common_df['label_pred'])
print("Precision:", precision)

recall = recall_score(common_df['label_df'], common_df['label_pred'])
print("Recall:", recall)

MCC = matthews_corrcoef(common_df['label_df'], common_df['label_pred'])
print("Matthews Correlation Coefficient:", MCC)

F1 Score: 0.7987987987987988
Accuracy: 0.7413127413127413
Precision: 0.7112299465240641
Recall: 0.910958904109589
Matthews Correlation Coefficient: 0.47940165834342563
